In [1]:
import pandas as pd

In [20]:
#read the date and show the table
df = pd.read_csv('../mvr-data.csv', parse_dates=['DateOfRecord', 'DateOfEvent'])
df

,DateOfRecord,DateOfEvent,Place,TypeOfRoad,Event,Vehicle1,Vehicle2,Outcome,AgeV1,AgeV2,TypeV2,Crossroad,Hour,GenderV1,GenderV2,AlchoholV2,Region,Mun
0,2015-05-28,2015-05-28,Тетово,Urban,кражба на велосипед,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Полошки,NaN
1,2015-02-06,2015-02-06,с.Враништа,Open,сообраќајна незгода,велосипед,автомобил,смрт,79.0,NaN,Фолксваген Пасат,не,11:00,NaN,NaN,NaN,Југозападен,Струга
2,2015-08-30,2015-08-29,Струга,Open,сообраќајна незгода,велосипед,автомобил,смрт,66.0,27.0,Мерцедес,не,16:00,NaN,NaN,NaN,Југозападен,NaN
3,2015-09-16,2015-09-15,Кичево,Open,сообраќајна незгода,велосипед,автомобил,смрт,74.0,44.0,Цитроен Берлинго,да,11:20,NaN,NaN,NaN,Југозападен,NaN
4,2015-09-19,2015-09-18,Скопје,Urban,сообраќајна незгода,велосипед,автомобил,смрт,13.0,47.0,Фиат Пунто,да,18:30,NaN,ж,NaN,Скопски,Центар
5,2015-09-22,2015-09-21,Свети Николе,Urban,напад на велосипедист,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17:45,NaN,NaN,NaN,Вардарски,NaN
6,2015-09-26,2015-09-25,Скопје,Urban,сообраќајна незгода,велосипед,автомобил,смрт,64.0,20.0,Ауди,да,14:00,NaN,NaN,NaN,Скопски,Центар
7,2015-12-14,2015-12-13,с.Долно Коњари,Rural,сообраќајна незгода,велосипед,автомобил,животна опасност,74.0,70.0,Опел Олимпија,не,15:30,NaN,NaN,NaN,Скопски,Петровец
8,2016-01-01,2015-12-31,Охрид,Urban,сообраќајна незгода,велосипед,автомобил,смрт,62.0,53.0,Фолксваген Пасат,не,17:00,NaN,NaN,да,Југозападен,NaN
9,2016-02-21,2016-02-20,с.Велешта,Open,сообраќајна незгода,велосипед со мотор,автобус,тешка повреда,52.0,45.0,Автобус,не,17:10,NaN,NaN,NaN,Југозападен,Струга


### R code for transformation

This should be rewritten in python:
this chunk makes a new df keeping only the traffic accinents (excluding other things like dog attacks), and adds new variables (columns) in order to have more fine-grained data points.


```
#keep the traffic accidents
traffacc <- accidents %>%
  #separate Date of event; having months as observations might be useful.
  separate(DateOfEvent, c("Day", "Month", "Year"), sep = "\\.", remove = FALSE) %>%
  #separate hours:mins, make numeric
  separate(Hour, c("H", "M"), sep = "\\:", remove = FALSE) %>%
  mutate(Hours = as.numeric(H)) %>%
  #filter so we keep just the traffic accidents
  filter(Event == "сообраќајна незгода") %>% 
  mutate(Outcome = ordered(Outcome, levels = c("непознато", "повреда", "тешка повреда", "животна опасност", "смрт")))
  
```

In [25]:
traffacc = df
traffacc['Year'] = pd.DatetimeIndex(df['DateOfEvent']).year
traffacc['Month'] = pd.DatetimeIndex(df['DateOfEvent']).month
traffacc['Day'] = pd.DatetimeIndex(df['DateOfEvent']).day
traffacc

,DateOfRecord,DateOfEvent,Place,TypeOfRoad,Event,Vehicle1,Vehicle2,Outcome,AgeV1,AgeV2,...,Crossroad,Hour,GenderV1,GenderV2,AlchoholV2,Region,Mun,Year,Month,Day
0,2015-05-28,2015-05-28,Тетово,Urban,кражба на велосипед,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Полошки,NaN,2015,5,28
1,2015-02-06,2015-02-06,с.Враништа,Open,сообраќајна незгода,велосипед,автомобил,смрт,79.0,NaN,...,не,11:00,NaN,NaN,NaN,Југозападен,Струга,2015,2,6
2,2015-08-30,2015-08-29,Струга,Open,сообраќајна незгода,велосипед,автомобил,смрт,66.0,27.0,...,не,16:00,NaN,NaN,NaN,Југозападен,NaN,2015,8,29
3,2015-09-16,2015-09-15,Кичево,Open,сообраќајна незгода,велосипед,автомобил,смрт,74.0,44.0,...,да,11:20,NaN,NaN,NaN,Југозападен,NaN,2015,9,15
4,2015-09-19,2015-09-18,Скопје,Urban,сообраќајна незгода,велосипед,автомобил,смрт,13.0,47.0,...,да,18:30,NaN,ж,NaN,Скопски,Центар,2015,9,18
5,2015-09-22,2015-09-21,Свети Николе,Urban,напад на велосипедист,NaN,NaN,NaN,NaN,NaN,...,NaN,17:45,NaN,NaN,NaN,Вардарски,NaN,2015,9,21
6,2015-09-26,2015-09-25,Скопје,Urban,сообраќајна незгода,велосипед,автомобил,смрт,64.0,20.0,...,да,14:00,NaN,NaN,NaN,Скопски,Центар,2015,9,25
7,2015-12-14,2015-12-13,с.Долно Коњари,Rural,сообраќајна незгода,велосипед,автомобил,животна опасност,74.0,70.0,...,не,15:30,NaN,NaN,NaN,Скопски,Петровец,2015,12,13
8,2016-01-01,2015-12-31,Охрид,Urban,сообраќајна незгода,велосипед,автомобил,смрт,62.0,53.0,...,не,17:00,NaN,NaN,да,Југозападен,NaN,2015,12,31
9,2016-02-21,2016-02-20,с.Велешта,Open,сообраќајна незгода,велосипед со мотор,автобус,тешка повреда,52.0,45.0,...,не,17:10,NaN,NaN,NaN,Југозападен,Струга,2016,2,20
